# Subscriptions Analysis
This notebook analyses subscriptions to different products

In [1]:
# Check Python version for compatibility/reference
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/Applications/JupyterLab.app/Contents/Resources/jlab_server/bin/python
3.8.12 | packaged by conda-forge | (default, Sep 16 2021, 01:59:00) 
[Clang 11.1.0 ]
sys.version_info(major=3, minor=8, micro=12, releaselevel='final', serial=0)


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Check Pandas and Numpy version numbering for compatibility/reference
print(f"{'Pandas version:'} \t{pd.__version__}")
print(f"{'NumPy version:'} \t\t{np.__version__}")

Pandas version: 	1.3.3
NumPy version: 		1.21.2


In [4]:
# Read the contents of the csv file into a Pandas dataframe
# Signal that the 'start', 'end', and 'cancelled' columns should be datetime objects
df = pd.read_csv('../data_files/subscriptions.csv',
                 parse_dates=['start','end','cancelled'],
                 infer_datetime_format=True)

In [5]:
# Check head of dataframe
df.head(10)

,start,end,cancelled,account_id,title,is_free
0,2016-01-25 10:26:00,2016-02-24 10:26:00,2016-02-23 17:05:00,581,Mapt - Pro [monthly],0
1,2016-02-23 17:09:00,2017-08-23 17:09:00,NaT,581,Mapt - Pro [18 Months],0
2,2014-12-03 09:38:00,2015-01-02 09:38:00,NaT,1071,Mapt - Pro [monthly],0
3,2015-01-02 01:39:00,2015-02-01 01:39:00,NaT,1071,Mapt - Pro [monthly],0
4,2015-02-01 01:48:00,2015-03-03 01:48:00,NaT,1071,Mapt - Pro [monthly],0
5,2015-03-03 01:39:00,2015-04-02 01:39:00,NaT,1071,Mapt - Pro [monthly],0
6,2015-04-02 01:42:00,2015-05-02 01:42:00,NaT,1071,Mapt - Pro [monthly],0
7,2015-05-02 01:29:00,2015-06-01 01:29:00,NaT,1071,Mapt - Pro [monthly],0
8,2015-06-01 01:33:00,2015-07-01 01:33:00,NaT,1071,Mapt - Pro [monthly],0
9,2015-07-01 01:46:00,2015-07-31 01:46:00,NaT,1071,Mapt - Pro [monthly],0


#### Data Quality and Data Cleansing

In [6]:
# Get information on data types and presence of null cancelled values
# Also confirms start', 'end', and 'cancelled' columns have datetime data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123783 entries, 0 to 123782
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   start       123783 non-null  datetime64[ns]
 1   end         123783 non-null  datetime64[ns]
 2   cancelled   15231 non-null   datetime64[ns]
 3   account_id  123783 non-null  int64         
 4   title       123783 non-null  object        
 5   is_free     123783 non-null  int64         
dtypes: datetime64[ns](3), int64(2), object(1)
memory usage: 5.7+ MB


In [7]:
# Convert 0 to False and 1 to True
df['is_free'] = np.where(df['is_free'] == 1, True, False)

# Verify change
df.head(5)

,start,end,cancelled,account_id,title,is_free
0,2016-01-25 10:26:00,2016-02-24 10:26:00,2016-02-23 17:05:00,581,Mapt - Pro [monthly],False
1,2016-02-23 17:09:00,2017-08-23 17:09:00,NaT,581,Mapt - Pro [18 Months],False
2,2014-12-03 09:38:00,2015-01-02 09:38:00,NaT,1071,Mapt - Pro [monthly],False
3,2015-01-02 01:39:00,2015-02-01 01:39:00,NaT,1071,Mapt - Pro [monthly],False
4,2015-02-01 01:48:00,2015-03-03 01:48:00,NaT,1071,Mapt - Pro [monthly],False


#### Interrogate the Dataset for Cleansing

In [8]:
# Confirm that all records contains a start and end date
df['start'].notnull().sum() == df['end'].notnull().sum() == df['start'].count()

True

In [9]:
# Get the earliest dated start date record
min(df['start'])

Timestamp('2013-01-01 01:41:00')

In [10]:
# Get the latest dated start date record
max(df['start'])

Timestamp('2021-10-07 19:18:00')

#### Calculate Some Headline Data

In [11]:
# Number of records
df['start'].count()

123783

In [12]:
# Number of Cancelled records
cancelled_recs = df['cancelled'].notnull().sum()
cancelled_recs

15231

In [13]:
# Number of Uncancelled records
uncancelled_recs = df['cancelled'].isnull().sum()
uncancelled_recs

108552

In [14]:
# Sanity check to ensure that the sum of uncancelled and cancelled records equals the total records
total_recs = cancelled_recs + uncancelled_recs
total_recs

123783

In [15]:
# Number of distinct accounts
df['account_id'].nunique()

40895

In [16]:
# Get row count by subscription title to get a sense of most/least common subscriptions
df['title'].value_counts()

Mapt - Pro [monthly]              83724
Mapt - Pro                        17407
Mapt Pro - 30 Day Free Trial      14229
Mapt - Basic [monthly]             2364
Mapt - Basic                       2363
Mapt - Pro [18 Months]             2185
Mapt Basic - 30 Day Free Trial     1511
Name: title, dtype: int64

In [17]:
# Sanity check to ensure that the sum of the title groupings equals the total records
df['title'].value_counts().sum() == total_recs

True

In [18]:
# Compile array of the dataset's records where the 'start' column is future-dated
from datetime import datetime
fd_recs = np.where(df['start'] > datetime.now())

In [19]:
# Count the number of numpy array elements where the 'start' column is future-dated
np.count_nonzero(fd_recs)

1

# START FROM HERE

## NEED TO WORK OUT HOW TO DROP THE 13 FUTURE_DATED ROWS FROM DATAFRAME

In [20]:
# Remove future-dated rows from the dataframe
# df = df.drop(fd_recs.index, axis=0)

In [21]:
# Check the number of records has now been reduced by 13
df['start'].count()

123783

#### Append Helper Columns to Analysis

In [22]:
# Add subscription type column to differentiate between 'Pro' and 'Basic' subscriptions
# This means that future subscription types could be added so long as their title included 'Pro'
df['sub_type'] = np.where(df['title'].str.contains('Pro'), 'Pro', 'Basic')

In [23]:
# Add subscription level column to differentiate between 'Trial' and 'Paid' subscriptions
df['sub_level'] = np.where(df['title'].str.contains('Trial'), 'Trial', 'Paid')

In [24]:
df['sub_duration (s)'] = np.where(df['cancelled'].isnull(), df['end'] - df['start'], df['cancelled'] - df['start'])

In [25]:
df.head(5)

,start,end,cancelled,account_id,title,is_free,sub_type,sub_level,sub_duration (s)
0,2016-01-25 10:26:00,2016-02-24 10:26:00,2016-02-23 17:05:00,581,Mapt - Pro [monthly],False,Pro,Paid,29 days 06:39:00
1,2016-02-23 17:09:00,2017-08-23 17:09:00,NaT,581,Mapt - Pro [18 Months],False,Pro,Paid,547 days 00:00:00
2,2014-12-03 09:38:00,2015-01-02 09:38:00,NaT,1071,Mapt - Pro [monthly],False,Pro,Paid,30 days 00:00:00
3,2015-01-02 01:39:00,2015-02-01 01:39:00,NaT,1071,Mapt - Pro [monthly],False,Pro,Paid,30 days 00:00:00
4,2015-02-01 01:48:00,2015-03-03 01:48:00,NaT,1071,Mapt - Pro [monthly],False,Pro,Paid,30 days 00:00:00


In [26]:
# Check data type of sub_duration helper column
df.dtypes['sub_duration (s)']

dtype('<m8[ns]')

In [27]:
# Convert sub_duration field into duration in seconds
df['sub_duration (s)'] = [td.total_seconds() for td in df['sub_duration (s)']]

In [28]:
df.head()

,start,end,cancelled,account_id,title,is_free,sub_type,sub_level,sub_duration (s)
0,2016-01-25 10:26:00,2016-02-24 10:26:00,2016-02-23 17:05:00,581,Mapt - Pro [monthly],False,Pro,Paid,2529540.0
1,2016-02-23 17:09:00,2017-08-23 17:09:00,NaT,581,Mapt - Pro [18 Months],False,Pro,Paid,47260800.0
2,2014-12-03 09:38:00,2015-01-02 09:38:00,NaT,1071,Mapt - Pro [monthly],False,Pro,Paid,2592000.0
3,2015-01-02 01:39:00,2015-02-01 01:39:00,NaT,1071,Mapt - Pro [monthly],False,Pro,Paid,2592000.0
4,2015-02-01 01:48:00,2015-03-03 01:48:00,NaT,1071,Mapt - Pro [monthly],False,Pro,Paid,2592000.0


In [29]:
# Create grouping table for each account and subscription level
# It seems appropriate to group on subscriptions which are trials versus those that are paid
sub_lengths_by_account = df.sort_values(['sub_level'], ascending=False).groupby(['account_id', 'sub_level']).agg({'sub_duration (s)':np.sum})

In [30]:
# Append a column that calculates the duration of the subscription in the number of days
sub_lengths_by_account['sub_duration (days)'] = sub_lengths_by_account['sub_duration (s)']/24/60/60

In [31]:
# Drop 'sub_duration (s)' column
sub_lengths_by_account = sub_lengths_by_account.drop(columns=['sub_duration (s)'])

In [32]:
# Round the duration in days to 2 d.p.
# Display first 20 rowssufficient to see examples of trial only, paid only, and trial and paid
sub_lengths_by_account.round(2).head(20)

,,sub_duration (days)
account_id,sub_level,
581,Paid,576.28
791,Paid,365.00
985,Paid,365.00
1020,Trial,30.00
1071,Paid,930.00
1081,Paid,1095.00
1130,Paid,365.00
1300,Paid,1095.00
1448,Paid,30.00
